Notebook for QLSTM 3 based on paper bibtex: Qi2021

In [1]:
import custom_networks as cn
import custom_circuits as cc
import torch
import utils.data_processing as dp
import pandas as pd

import torch.nn as nn
import torchmetrics

In [2]:
import warnings

# ignore off UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# Data parameters:
# DATA_PATH = "data/data_damped_oscillator.csv"
DATA_PATH = "data/custom_created_pattern.csv"
TIME_SEQUENCE = 4

# Model parameters:
INPUT_SIZE = 1 # the input feature dimension, in our case with functions always 1
HIDDEN_SIZE = 1 #dimension of the hidden state, represents the size or dimensionality of the hidden state in the LSTM

# Training parameters:
TRAIN_TEST_SPLIT = 0.67
NUM_EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 0.01

FEATURE_MAP = cc.FeatureMap_3()
# the same vlayer as for paper 2
ANSATZ = cc.VariationalLayer_2()
PAPER = "paper_3"

REPS = 2

TEST_RUNS = 30

In [4]:
# load the data
data = pd.read_csv(DATA_PATH, usecols=["y"])

# preprocess and split data - data_processing() from data_processing.py
input_train, target_train, input_test, target_test = dp.data_processing(data, time_sequence=TIME_SEQUENCE, train_test_split=TRAIN_TEST_SPLIT, reshape_inputs=True)

# create a TensorDataset from input and target tensors
train_dataset = torch.utils.data.TensorDataset(input_train, target_train)

# create a DataLoader for efficient batch processing
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
# generator and worker_init_fn is used to assure reproducibility (https://pytorch.org/docs/stable/notes/randomness.html Section: DataLoader)


# reshape the the target(_test & _train) tensor [len, 1, 1] - to match the shape of the pred tensor [len, 1]
target_test_reshape = target_test.reshape(len(target_test), 1)
target_train_reshape = target_train.reshape(len(target_train), 1)

All shapes are correct.


In [5]:
for run in range(TEST_RUNS):
    print(f"\nRun [{run+1}/{TEST_RUNS}]")
    run_metrics = {"model_name": "QLSTM", "data": DATA_PATH, "epochs": NUM_EPOCHS, "Reps": REPS, "FeatureMap": PAPER, "Ansatze": PAPER}

    # initialize the model
    q_lstm = cn.QuantumLongShortTermMemory(FEATURE_MAP, ANSATZ, REPS)

    # initialize the loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(q_lstm.parameters(), lr=LEARNING_RATE)

    # set the model to training mode
    q_lstm.train() 

    # training loop
    losses = []

    # get the number of batches to process
    if len(train_dataset) % BATCH_SIZE == 0:
        num_batches = len(train_dataset) // BATCH_SIZE
    else:
        num_batches = len(train_dataset) // BATCH_SIZE + 1

    for epoch in range(NUM_EPOCHS):
        # clear gradients them out before each instance
        q_lstm.zero_grad()

        running_loss = 0.0
        batch_num = 1
        for batch_input, batch_target in data_loader:
            print(f"\rEpoch [{epoch+1}/{NUM_EPOCHS}]: Batch Number [{batch_num}/{num_batches}]", end="", flush=True)
            batch_num += 1
            # zero the gradients
            optimizer.zero_grad()

            # forward pass
            output , (_, _) = q_lstm(batch_input)
            # compute the loss
            loss = loss_function(output, batch_target.reshape(len(batch_target), 1))

            # backward pass
            loss.backward()

            # update the weights
            optimizer.step()

            # accumulate the loss
            running_loss += loss.item()

        # compute the average loss for the epoch
        epoch_loss = running_loss / len(data_loader)

        # save the loss for the epoch
        run_metrics[f"loss_epoch_{epoch+1}"] = epoch_loss

    # get evaluation metrics
    # set the model to evaluation mode
    q_lstm.eval()  

    # predict the test set
    with torch.no_grad():
        ### prediction for the test set
        pred_test, (_, _) = q_lstm(input_test)
        # calculate errors
        pred_test = pred_test.reshape(len(target_test_reshape), 1)

        mean_squared_error = nn.MSELoss()
        mse_test = mean_squared_error(pred_test, target_test_reshape)

        mean_abs_error = torchmetrics.MeanAbsoluteError()
        mae_test = mean_abs_error(pred_test, target_test_reshape)

        ### prediction for the train set
        pred_train, (_, _) = q_lstm(input_train)

        mean_squared_error = nn.MSELoss()
        pred_train = pred_train.reshape(len(target_train_reshape), 1)
        mse_train = mean_squared_error(pred_train, target_train_reshape)

        mean_abs_error = torchmetrics.MeanAbsoluteError()
        mae_train = mean_abs_error(pred_train, target_train_reshape)

    run_metrics["mse_train"] = mse_train.item()
    run_metrics["mse_test"] = mse_test.item()
    run_metrics["mae_train"] = mae_train.item()
    run_metrics["mae_test"] = mae_test.item()

    # save metrics for the run
    metrics = pd.read_csv("data/qlstm_metrics.csv")
    # Append the new row to the DataFrame
    metrics = pd.concat([metrics, pd.DataFrame([run_metrics])], ignore_index=True)
    # save metrics
    metrics.to_csv("data/qlstm_metrics.csv", index=False)


Run [1/30]
Epoch [1/30]: Batch Number [1/13]

Epoch [30/30]: Batch Number [13/13]
Run [2/30]
Epoch [30/30]: Batch Number [13/13]
Run [3/30]
Epoch [30/30]: Batch Number [13/13]
Run [4/30]
Epoch [30/30]: Batch Number [13/13]
Run [5/30]
Epoch [30/30]: Batch Number [13/13]
Run [6/30]
Epoch [30/30]: Batch Number [13/13]
Run [7/30]
Epoch [30/30]: Batch Number [13/13]
Run [8/30]
Epoch [30/30]: Batch Number [13/13]
Run [9/30]
Epoch [30/30]: Batch Number [13/13]
Run [10/30]
Epoch [30/30]: Batch Number [13/13]
Run [11/30]
Epoch [30/30]: Batch Number [13/13]
Run [12/30]
Epoch [30/30]: Batch Number [13/13]
Run [13/30]
Epoch [30/30]: Batch Number [13/13]
Run [14/30]
Epoch [30/30]: Batch Number [13/13]
Run [15/30]
Epoch [30/30]: Batch Number [13/13]
Run [16/30]
Epoch [26/30]: Batch Number [4/13]]